# Scraping datos balance anuales investing

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import pandas as pd
from bs4 import BeautifulSoup




def get_balance_sheet(tk):
    dfs = []
    # Configurar el navegador
    # chrome_options = Options()
    # chrome_options.add_argument("--headless")
    driver = webdriver.Chrome()
    try:
        # Abrir la URL
        url = f"https://es.investing.com/equities/{tk}-balance-sheet"
        driver.get(url)

        # Esperar a que al menos una tabla esté presente
        
        wait = WebDriverWait(driver, 5)
        wait.until(EC.presence_of_element_located((By.TAG_NAME, "table")))

        # Obtener el HTML de la página
        page_html = driver.page_source

        # Parsear el HTML de la página con BeautifulSoup
        soup = BeautifulSoup(page_html, "html.parser")

        # Encontrar todas las tablas en la página
        tables = soup.find_all("table")

        # Procesar cada tabla
        for i, table in enumerate(tables):
            # Extraer las filas de la tabla
            rows = table.find_all("tr")
            data = []
            for row in rows:
                cells = row.find_all(["td", "th"])  # Buscar celdas (td) y encabezados (th)
                cell_texts = [cell.get_text(strip=True) for cell in cells]
                data.append(cell_texts)

            # Convertir los datos en un DataFrame de Pandas
            df = pd.DataFrame(data)
            dfs.append(df)
        print("exito al procesar!")

    except TimeoutException:
        print("Error: No se encontraron tablas en la página.")
    except Exception as e:
        print(f"Error: {e}")
    finally:
        # Cerrar el navegador
        driver.quit()
        time.sleep(5)
    return dfs

#### Procesamiento de datos capturados

In [2]:
def procesar_tab(df):
  titulos = df[1].T[0].dropna().reset_index(drop=True).tolist()[1:]
  titulos[0] = "indicador"

  result = pd.DataFrame()
   
  estructura = [
  {"seccion":"activo_corriente", "rubros":2,"valores":3},
  {"seccion":"activo_total", "rubros":4,"valores":5},
  {"seccion":"pasivo_corriente", "rubros":6,"valores":7},
  {"seccion":"pasivo_total", "rubros":8,"valores":9},
  {"seccion":"patrimonio_neto", "rubros":10,"valores":11},
  {"seccion":"crecimiento_deuda", "rubros":14,"valores":15},
]
  for e in estructura:
    try:
      data =  pd.concat([df[e["rubros"]][1],df[e["valores"]]], axis=1)
      data.columns = titulos
      data.replace("aa.aa",None,inplace=True)
      data.dropna(inplace=True, axis=1)
      data["seccion"] = e["seccion"]
      result = pd.concat([result, data.reset_index(drop=True)], axis=0)
    except:
      pass
    
  return result.set_index("indicador")

### Lista de Empresas

In [3]:
companies = [
  #  "apple-computer-inc",
  # "microsoft-corp",
  # "google-inc",
  # "tesla-motors",
  # "visa-inc",
  # "berkshire-hathaway-inc",
  # "johnson-johnson",
  # "pfizer",
  # "amazon-com-inc",
  # "disney",
  "nike",
  # "procter-gamble",
  # "coca-cola-co",
  # "chevron",
  # "3m-co"
]

In [ ]:
for company in companies:
  print(f"Procesando {company}")
  dfs = get_balance_sheet(company)
  if len(dfs) > 1:
    df = procesar_tab(dfs)
    df.to_csv(f"balance-sheet/{company}.csv",encoding="utf-8-sig")
    print(f"Guardado {company}.csv")
  else:
    print(f"Error al procesar {company}")